In [2]:
import torch
inputs = torch.tensor([
    [0.43, 0.15, 0.89],
    [0.22, 0.88, 0.33],
    [0.57, 0.12, 0.99],
    [0.71, 0.23, 0.44],
    [0.11, 0.54, 0.77],
    [0.98, 0.36, 0.22]
])

query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(query, x_i)
print(attn_scores_2)
attn_weights_2_temp = attn_scores_2 / attn_scores_2.sum()
print(attn_weights_2_temp)

tensor([0.5203, 0.9317, 0.5577, 0.5038, 0.7535, 0.6050])
tensor([0.1344, 0.2406, 0.1440, 0.1301, 0.1946, 0.1562])


In [4]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)
print(attn_weights_2_naive)
print(attn_weights_2_naive.sum())

tensor([0.1453, 0.2193, 0.1509, 0.1429, 0.1835, 0.1582])
tensor(1.0000)


In [7]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
context_vector_2 = torch.zeros(query.shape[0])
for i, x_i in enumerate(inputs):
    context_vector_2 += attn_weights_2[i] * x_i
print(context_vector_2)

tensor([0.4734, 0.4217, 0.5900])


In [12]:
attn_scores = torch.empty(inputs.shape[0], inputs.shape[0])
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
print(attn_scores)

attn_weights = torch.softmax(attn_scores, dim=1)
print(attn_weights)

tensor([[0.9995, 0.5203, 1.1442, 0.7314, 0.8136, 0.6712],
        [0.5203, 0.9317, 0.5577, 0.5038, 0.7535, 0.6050],
        [1.1442, 0.5577, 1.3194, 0.8679, 0.8898, 0.8196],
        [0.7314, 0.5038, 0.8679, 0.7506, 0.5411, 0.8754],
        [0.8136, 0.7535, 0.8898, 0.5411, 0.8966, 0.4716],
        [0.6712, 0.6050, 0.8196, 0.8754, 0.4716, 1.1384]])
tensor([[0.1965, 0.1217, 0.2270, 0.1503, 0.1631, 0.1415],
        [0.1453, 0.2193, 0.1509, 0.1429, 0.1835, 0.1582],
        [0.1998, 0.1112, 0.2381, 0.1516, 0.1549, 0.1444],
        [0.1682, 0.1340, 0.1929, 0.1715, 0.1391, 0.1943],
        [0.1792, 0.1688, 0.1934, 0.1365, 0.1947, 0.1273],
        [0.1484, 0.1389, 0.1722, 0.1821, 0.1216, 0.2368]])


In [14]:
# Faster approach
attn_scores = inputs @ inputs.T
attn_weights = torch.softmax(attn_scores, dim=-1)
all_context_vectors = attn_weights @ inputs

# Now with trainable weights

In [24]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
w_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

x2_query = x_2 @ W_query
x2_key = x_2 @ W_key
x2_value = x_2 @ w_value

keys = inputs @ W_key
values = inputs @ w_value

keys2 = keys[1]
attn_scores_22 = x2_query.dot(keys2)
attn_scores_2 = x2_query @ keys.T 
print(attn_scores_2)
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k ** 0.5, dim=-1)
context_vector_2 = attn_weights_2 @ values
print(context_vector_2)

tensor([0.8829, 0.9856, 0.9585, 0.6316, 1.0489, 0.5998])
tensor([0.2340, 0.7687])
